In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
l1_keywords_frame = pd.read_csv("/content/drive/MyDrive/T4SG St Jude/Asset CSVs/L1Groupings.csv")

n = 61
img_name = l1_keywords_frame.iloc[n, 1]
keywords = l1_keywords_frame.iloc[n, 3]

print('Image name: {}'.format(img_name))
print(keywords)

Image name: 10002670-124-V1_TIF.jpg
Campus, Decorations


In [ ]:
class Level1KeywordsDataset(Dataset):
    """Keywords dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.keywords_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.keywords_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.keywords_frame.iloc[idx, 1])
        image = io.imread(img_name)
        keywords = self.keywords_frame.iloc[idx, 3].split(", ")
        keyword_map = {'People': 0,
                       'Drone - Aerial': 1,
                       'Campus': 2,
                       'Science': 3,
                       'Labs': 4,
                       'Memphis': 5,
                       'Events': 6,
                       'Decorations': 7,
                       'St Jude Research Website': 8,
                       'Clinical': 9,
                       'Board of Governors': 10,
                       'Convocation': 11,
                       'Covid': 12,
                       'ALSAC': 13}
        keywords_arr = np.zeros(14, dtype=float)
        for i in keywords:
            if i in keyword_map.keys():
                keywords_arr[keyword_map[i]] = 1
        keywords = keywords_arr
        sample = {'image': image, 'keywords': keywords}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
import skimage

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, keywords = sample['image'], sample['keywords']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = skimage.transform.resize(image, (new_h, new_w))

        # h and w are swapped for keywords because for images,
        # x and y axes are axis 1 and 0 respectively

        return {'image': img, 'keywords': keywords}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, keywords = sample['image'], sample['keywords']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h + 1)
        left = np.random.randint(0, w - new_w + 1)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'keywords': keywords}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, keywords = sample['image'], sample['keywords']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        # keywords = keywords.split(", ")
        # for i in range(len(keywords)):
        #     keywords[i] = keyword_map[i]
        # keywords = np.array(keywords)
        # print(image.size)
        return {'image': torch.from_numpy(image),
                'keywords': torch.from_numpy(keywords)}

In [ ]:
transform = transforms.Compose([
    Rescale((256, 256)),
    ToTensor()
])

batch_size = 1
root_dir = "/content/drive/MyDrive/T4SG St Jude/Labelled Assets"
csv_file = "/content/drive/MyDrive/T4SG St Jude/Asset CSVs/L1Groupings.csv"
test_file = "/content/drive/MyDrive/T4SG St Jude/Asset CSVs/L1GroupingsPart5.csv"

trainset = Level1KeywordsDataset(csv_file=csv_file,
                                 root_dir=root_dir,
                                 transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0)

testset = Level1KeywordsDataset(csv_file=test_file,
                                 root_dir=root_dir,
                                 transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=0)

classes = ('People', 'Drone - Aerial', 'Campus', 'Science', 'Labs', 'Memphis',
           'Events','Decorations','St Jude Research Website','Clinical',
           'Board of Governors','Convocation','Covid','ALSAC')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(61 * 61 * 16, 256) # Initial Layer
        self.fc2 = nn.Linear(256, 512) # Increase # of neurons: more things to learn
        # self.fc3 = nn.Linear(512, 1024) # Optional (paired w/ below) (MODEL PERFORMS BETTER WITHOUT IT)
        # self.fc4 = nn.Linear(1024, 512) # Optional (paired w/ above) (MODEL PERFORMS BETTER WITHOUT IT)
        self.fc5 = nn.Linear(512, 256) # Decrease to converge to image
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 14)
        self.double() # Needed otherwise the types break

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['image'], data['keywords']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: 0.367
[1,    40] loss: 0.285
[1,    60] loss: 0.342
[1,    80] loss: 0.311
[1,   100] loss: 0.306
[1,   120] loss: 0.296
[1,   140] loss: 0.372
[1,   160] loss: 0.266
[2,    20] loss: 0.447
[2,    40] loss: 0.267
[2,    60] loss: 0.260
[2,    80] loss: 0.348
[2,   100] loss: 0.283
[2,   120] loss: 0.368
[2,   140] loss: 0.352
[2,   160] loss: 0.274
[3,    20] loss: 0.305
[3,    40] loss: 0.217
[3,    60] loss: 0.367
[3,    80] loss: 0.260
[3,   100] loss: 0.256
[3,   120] loss: 0.324
[3,   140] loss: 0.368
[3,   160] loss: 0.395
[4,    20] loss: 0.323
[4,    40] loss: 0.250
[4,    60] loss: 0.347
[4,    80] loss: 0.348
[4,   100] loss: 0.279
[4,   120] loss: 0.331
[4,   140] loss: 0.309
[4,   160] loss: 0.341
[5,    20] loss: 0.327
[5,    40] loss: 0.329
[5,    60] loss: 0.358
[5,    80] loss: 0.297
[5,   100] loss: 0.325
[5,   120] loss: 0.294
[5,   140] loss: 0.275
[5,   160] loss: 0.356
[6,    20] loss: 0.351
[6,    40] loss: 0.305
[6,    60] loss: 0.340
[6,    80] 

In [ ]:
input("PROCEED IF YOU ARE VERY SURE YOU WANT TO SAVE A MODEL")

PATH = "/content/drive/MyDrive/T4SG St Jude/20EpochTrainedClassifierv2.pth"
torch.save(net.state_dict(), PATH)

PROCEED IF YOU ARE VERY SURE YOU WANT TO SAVE A MODEL


# Testing

In [ ]:
input("PROCEED IF YOU ARE VERY SURE YOU WANT TO LOAD A MODEL")

net = Net()
PATH = "/content/drive/MyDrive/T4SG St Jude/20EpochTrainedClassifier.pth"
net.load_state_dict(torch.load(PATH, weights_only=True))

PROCEED IF YOU ARE VERY SURE YOU WANT TO LOAD A MODEL


<All keys matched successfully>

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)['image'], next(dataiter)['keywords']
outputs = net(images)
print(labels)

tensor([[0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64)


In [ ]:
_, predicted = torch.max(outputs, 1)
print(classes[predicted[0]])

Campus


In [ ]:
pred_sum = 0
control_sum = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data['image'], data['keywords']
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        # print(outputs)
        predicted = outputs.flatten().numpy()
        max_val = np.max(predicted)
        min_val = np.min(predicted)
        predicted = (predicted - min_val) / (max_val - min_val)
        predicted = np.where(predicted >= 0.75, 1, 0)
        predicted_keywords = [keyword for keyword in classes if predicted[classes.index(keyword)] == 1]
        print(predicted_keywords)
        labels = labels.flatten().numpy()
        label_keywords = [keyword for keyword in classes if labels[classes.index(keyword)] == 1]
        print(label_keywords)

        cosine_similarity = np.dot(predicted, labels) / (np.linalg.norm(predicted) * np.linalg.norm(labels))
        # print(cosine_similarity)
        pred_sum += cosine_similarity
        control_group = np.array([0,0,1,1,1,0,0,0,0,0,0,0,0,0]) # Campus, Science, Labs
        control_similarity = np.dot(labels, control_group) / (np.linalg.norm(labels) * np.linalg.norm(control_group))
        # print(control_similarity)
        control_sum += control_similarity
        # print("\n")
        # print(labels)
        # total += labels.size(0
        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()

print(pred_sum / len(testloader))
print(control_sum / len(testloader))

['Campus', 'Science', 'Labs']
['People']
['Campus']
['Drone - Aerial', 'Campus']
['Campus', 'Science', 'Labs']
['Science', 'Labs']
['Campus']
['People', 'Science', 'Labs']
['Campus']
['Campus', 'Labs']
['Campus']
['Drone - Aerial', 'Campus', 'Labs']
['Campus', 'Science', 'Labs']
['Memphis']
['Campus']
['Drone - Aerial', 'Campus']
['Campus', 'Science', 'Labs']
['People', 'Science', 'Labs']
['Campus']
['Memphis']
['Campus', 'Science', 'Labs']
['People', 'Labs', 'St Jude Research Website']
['Campus', 'Science', 'Labs']
['Science', 'Labs', 'St Jude Research Website']
['Campus']
['People', 'Science']
['Campus', 'Science', 'Labs']
['Science', 'Labs', 'St Jude Research Website']
['People', 'Campus', 'Science', 'Labs']
['People', 'Science']
['Campus', 'Science', 'Labs']
['People', 'Science', 'Labs']
['Campus', 'Labs']
['Memphis']
['Campus']
['Campus', 'ALSAC']
['Campus']
['People', 'Events', 'Convocation']
['Campus', 'Science', 'Labs']
['Campus', 'Decorations']
['People', 'Science', 'Labs']
['